In [33]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import pi
# qiskit
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)
# qurry
sys.path.insert(0, './')
from backend import backendWrapper
from mori import TagMap
print("Modules import completed...")
print("-"*30+"\n ### Init IBMQ and Set up provider")
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(
    hub='ibm-q-hub-ntu', group='ntu-internal', project='default')
backend = backendWrapper(provider)
print("IBMQ loading completed...")

Modules import completed...
------------------------------
 ### Init IBMQ and Set up provider
IBMQ loading completed...


In [34]:
qaoaQurry = Qurry()
qaoaName = 'tsp'
problesSize = 2

nqubits = 3 ** 2
sampling = 10
shot = 2**(nqubits+6)
print(shot)

32768


In [35]:
from typing import Callable
from collections import defaultdict

def H_Ising(bitstring, Ising_coeff):
    spins = []
    for bit in bitstring:
        if bit == '0':
            spins.append(-1)
        elif bit == '1':
            spins.append(1)
    H = 0
    for i in range(0, nqubits):
        H += Ising_coeff[i][i]*spins[i]
        for j in range(i+1, nqubits):
            H += Ising_coeff[i][j]*spins[i]*spins[j]
    return H

def compute_expectation(counts: dict[str, int], Ising_coeff: np.ndarray):   
    """
    Computes expectation value based on measurement results
    """   
    sum = 0
    sum_count = 0
    for bitstring, count in counts.items():
        sum += count*H_Ising(bitstring, Ising_coeff)
        sum_count += count
    return sum/sum_count

from IPython.display import clear_output


# We will also bring the different circuit components that
# build the qaoa circuit under a single function
def create_qaoa_circ(
    Ising_coeff: np.ndarray, 
    theta: list[float],
) -> QuantumCircuit: 
    """
    Creates a parametrized qaoa circuit
    """   
    p = len(theta)//2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)
    
    beta = theta[:p]
    gamma = theta[p:]
    
    # initial_state
    for i in range(0, nqubits):
        qc.h(i)
    
    for irep in range(0, p):
        
        # problem unitary
        for i in range(nqubits):
            for j in range(nqubits):
                if Ising_coeff != 0 and i!=j:
                    qc.rzz(2 * Ising_coeff[i][j]*gamma[irep], i, j)
                    
        for i in range(nqubits):
            qc.rz(2 * Ising_coeff[i][i]*gamma[irep], i)
        
        # mixer unitary
        for i in range(nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc

def qaoa_circs(
    Ising_coeff: np.ndarray,
) -> Callable[[list[float]], QuantumCircuit]:
    
    def theta_depend(theta: list[float]) -> QuantumCircuit:
        return create_qaoa_circ(
            Ising_coeff=Ising_coeff, 
            theta=theta,
        )
    
    return theta_depend

def expect(
    Ising_coeff: np.ndarray,
) -> Callable[[dict[str, int]], float]:
    
    def counts_depend(counts: dict[str, int]) -> float:
        return compute_expectation(
            counts=counts, 
            Ising_coeff=Ising_coeff,
        )
    
    return counts_depend

def countCombine(countList: list[dict[str, int]]) -> dict[str, int]:
    summary = defaultdict(int)
    for c in countList:
        for k, v in c.items():
            summary[k] += v
    return summary

def expecting(Ising_matrix: np.ndarray) -> Callable[[list[float]], float]:
    
    qaoa_circs_giver = qaoa_circs(Ising_matrix)
    expectation_giver = expect(Ising_matrix)
    # Finally we write a function that executes the circuit on the chosen backend
    def get_expectation(theta: list[float]) -> float:
        """
        Runs parametrized circuit and get expectation value
    
        """
        # waveName = qaoaQurry.addWave(qaoa_circs_giver(theta))
        # singleIterResult = qaoaQurry.multiOutput(
        #     configList=[{ 
        #         'wave': waveName,
        #         'tags': waveName,
        #         'shots': 2**(nqubits+3),
        #         'sampling': sampling,
        #     }],
        #     backend=backend('aer_gpu'),
        #     saveLocation=Path('test.004')
        # )
        # countsList: list[dict[str, int]] = singleIterResult['tagMapCounts'][waveName]
        # counts = countCombine(countsList)
        
        qc = qaoa_circs_giver(theta)
        counts = backend('aer_gpu').run(qc, shots=2**16).result().get_counts()
        clear_output()
        print(counts)
        global ccc
        ccc = counts

        return expectation_giver(counts)
    
    return get_expectation

In [36]:
ccc = {}

In [37]:
aaaa = [[10616.601717798214,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  2.5,
  0.0],
 [3535.533905932738,
  10616.601717798214,
  3535.533905932738,
  0.0,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  2.5],
 [3535.533905932738,
  3535.533905932738,
  10616.601717798214,
  2.5,
  0.0,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738],
 [3535.533905932738,
  2.5,
  0.0,
  10618.672785610077,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  3.5355339059327378,
  0.0],
 [0.0,
  3535.533905932738,
  2.5,
  3535.533905932738,
  10618.672785610077,
  3535.533905932738,
  0.0,
  3535.533905932738,
  3.5355339059327378],
 [2.5,
  0.0,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  10618.672785610077,
  3.5355339059327378,
  0.0,
  3535.533905932738],
 [3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  3.5355339059327378,
  0.0,
  10618.672785610077,
  3535.533905932738,
  3535.533905932738],
 [0.0,
  3535.533905932738,
  2.5,
  0.0,
  3535.533905932738,
  3.5355339059327378,
  3535.533905932738,
  10618.672785610077,
  3535.533905932738],
 [2.5,
  0.0,
  3535.533905932738,
  3.5355339059327378,
  0.0,
  3535.533905932738,
  3535.533905932738,
  3535.533905932738,
  10618.672785610077]]

In [38]:
import torch

torch.manual_seed(0)
theta = torch.empty(int(nqubits)*2)
torch.nn.init.normal_(theta, mean=np.pi, std=0.01*np.pi)
theta = theta.tolist()
theta

[3.1062233448028564,
 3.1053903102874756,
 3.1593945026397705,
 3.1665215492248535,
 3.1604058742523193,
 3.092737913131714,
 3.130868673324585,
 3.1998066902160645,
 3.1562983989715576,
 3.1366381645202637,
 3.1361453533172607,
 3.1473569869995117,
 3.1852409839630127,
 3.1914288997650146,
 3.1713216304779053,
 3.1150877475738525,
 3.1708669662475586,
 3.181145668029785]

In [55]:
from scipy.optimize import minimize

expectation = expecting(Ising_matrix=aaaa)

res = minimize(expectation, 
                theta, options ={'iprint':1, 'disp':False, 'maxiter':10000,
                                 'catol':0.002, 'rhobeg':5.0},
            method='COBYLA')
res

{'100100110': 1, '010010110': 2, '101000011': 3, '000011110': 2, '110000101': 1, '111010010': 4, '100111111': 4, '110111111': 1, '111010001': 2, '010110111': 6, '000101011': 2, '001011110': 8, '111001111': 5, '111000111': 5, '010111010': 10, '011011111': 9, '111111011': 2, '001111011': 5, '111001110': 7, '100110101': 12, '011000101': 1, '101101111': 3, '011001111': 6, '110100110': 14, '110011101': 13, '110010011': 11, '111001010': 8, '101001110': 2, '000100111': 4, '011011011': 7, '010111011': 7, '101100111': 4, '001001011': 4, '110100111': 5, '110110011': 4, '011010101': 4, '011100011': 17, '110101111': 7, '011011010': 8, '100101110': 16, '111001011': 10, '111100011': 10, '000101110': 4, '110110111': 5, '111010111': 10, '101101101': 9, '111100100': 6, '010111111': 4, '111011111': 2, '111101110': 11, '110010111': 3, '000010111': 3, '011010010': 27, '011111110': 8, '100100111': 10, '011100111': 13, '001001111': 6, '000101111': 5, '101011011': 20, '010011111': 8, '011110101': 18, '101111

     fun: -31178.832313510597
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 170
  status: 1
 success: True
       x: array([8.10925447, 3.22208742, 3.1439113 , 3.13348564, 3.14359837,
       3.11219026, 3.11529214, 3.2116978 , 3.12887284, 3.15200216,
       3.12514276, 3.1246901 , 3.15226629, 3.2171216 , 3.18979418,
       3.12408961, 3.18073589, 8.11393103])

In [56]:
def filterCounts(counts: dict[str, int]) -> dict[str, int]:
    bitLen = len(list(counts)[0])
    qubitLen = int(np.sqrt(bitLen))
    if (qubitLen % 1 > 0):
        raise ValueError(f"bitLen: {bitLen}, qubitLen: {qubitLen}.")

    def filter(bitstr: str) -> bool:
        strList = [bitstr[qubitLen*(k):qubitLen*(k+1)] for k in range(qubitLen)]
        strList1Sum = [sum([int(b) for b in bs]) for bs in strList]
        strList1Pos = [[i for i, b in enumerate(bs) if b == '1'] for bs in strList]
        # print(strList1Pos)
        strList1PosOnly = all([len(lbs) == 1 for lbs in strList1Pos])
        # print(strList1Pos, strList1PosOnly)
        strList1PosImmutable = sum([sum(l) for l in strList1Pos]) == (qubitLen-1)*qubitLen/2
        # print(strList1PosImmutable)
            
        return all([ s==1 for s in strList1Sum]) and strList1PosImmutable
    
    filteredCounts = {
        k: v
    for k, v in counts.items() if filter(k) }
    
    return filteredCounts

In [57]:
expectation

<function __main__.expecting.<locals>.get_expectation(theta: list) -> float>

In [58]:
sum(ccc.values()), sum(filterCounts(ccc).values()), sum(filterCounts(ccc).values())/sum(ccc.values())

(65536, 2438, 0.037200927734375)

In [59]:
def maxCounts(_counts: dict[str, int]) -> str:
    vs = list(_counts.values())
    vsMax = max(_counts.values())
    vsMaxIndex = list(_counts.keys())[vs.index(vsMax)]
    return vsMaxIndex

In [60]:
maxCounts(ccc)

'000000000'

In [61]:
ccc[maxCounts(ccc)]

5038

In [62]:
ccc

{'100100110': 1,
 '010010110': 2,
 '101000011': 3,
 '000011110': 2,
 '110000101': 1,
 '111010010': 4,
 '100111111': 4,
 '110111111': 1,
 '111010001': 2,
 '010110111': 6,
 '000101011': 2,
 '001011110': 8,
 '111001111': 5,
 '111000111': 5,
 '010111010': 10,
 '011011111': 9,
 '111111011': 2,
 '001111011': 5,
 '111001110': 7,
 '100110101': 12,
 '011000101': 1,
 '101101111': 3,
 '011001111': 6,
 '110100110': 14,
 '110011101': 13,
 '110010011': 11,
 '111001010': 8,
 '101001110': 2,
 '000100111': 4,
 '011011011': 7,
 '010111011': 7,
 '101100111': 4,
 '001001011': 4,
 '110100111': 5,
 '110110011': 4,
 '011010101': 4,
 '011100011': 17,
 '110101111': 7,
 '011011010': 8,
 '100101110': 16,
 '111001011': 10,
 '111100011': 10,
 '000101110': 4,
 '110110111': 5,
 '111010111': 10,
 '101101101': 9,
 '111100100': 6,
 '010111111': 4,
 '111011111': 2,
 '111101110': 11,
 '110010111': 3,
 '000010111': 3,
 '011010010': 27,
 '011111110': 8,
 '100100111': 10,
 '011100111': 13,
 '001001111': 6,
 '000101111': 5,


In [1]:
import numpy as np

In [16]:
def filterCounts(counts: dict[str, int]) -> dict[str, int]:
    bitLen = len(list(counts)[0])
    qubitLen = int(np.sqrt(bitLen))
    if (qubitLen % 1 > 0):
        raise ValueError(f"bitLen: {bitLen}, qubitLen: {qubitLen}.")

    def filter(bitstr: str) -> bool:
        strList = [bitstr[qubitLen*(k):qubitLen*(k+1)] for k in range(qubitLen)]
        strList1Sum = [sum([int(b) for b in bs]) for bs in strList]
        strList1PosImmutable = dict.fromkeys(strList)
            
        return all([ s==1 for s in strList1Sum]) and len(strList1PosImmutable) == 3
    
    filteredCounts = {
        k: v
    for k, v in counts.items() if filter(k) }
    
    return filteredCounts

filterCounts(ccccc)

{'010100001': 379,
 '010001100': 383,
 '001010100': 403,
 '100001010': 429,
 '001100010': 398,
 '100010001': 379}

In [15]:
filterCounts({'010010010': 33})

{}